In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
rf = joblib.load(r'C:\Users\devin\AI-agriculture-yield-prediction\Models\rf_model.pkl')
scaler = joblib.load(r'C:\Users\devin\AI-agriculture-yield-prediction\Models\scaler.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\devin\\AI-agriculture-yield-prediction\\Models\\rf_model.pkl'

In [ ]:
df = pd.read_csv(r'C:\Users\devin\AI-agriculture-yield-prediction\Datasets\test_predictions.csv')

In [ ]:
r2 = r2_score(df['Actual'], df['Predicted'])
mae = mean_absolute_error(df['Actual'], df['Predicted'])
rmse = mean_squared_error(df['Actual'], df['Predicted'])
print(r2*100) 
print(mae)
print(rmse)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6,5))
sns.scatterplot(x=df['Actual'], y=df['Predicted'])
plt.xlabel("Actual Yield (kg/ha)")
plt.ylabel("Predicted Yield (kg/ha)")
plt.title("Actual vs Predicted Rice Yield")
plt.savefig(r'C:\Users\devin\AI-agriculture-yield-prediction\Plots\actual_vs_predicted.png')
plt.grid(True)
plt.show()


In [ ]:
residuals = df['Actual'] - df['Predicted']
sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution")
plt.xlabel("Prediction Error")
plt.savefig(r'C:\Users\devin\AI-agriculture-yield-prediction\Plots\Prediction_error.png')
plt.show()